In [1]:
import pandas as pd
import numpy as np
import sys
import re
import os
import pandas as pd
sys.path.append("../automaterialsdata/")

from table_extractor import get_all_files_with_extension
from table_extractor import get_tables_from_xml_file
from table_extractor import gather_properties_from_full_table

In [2]:
Elsevier_xmls = get_all_files_with_extension('/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml', 'xml')
Elsevier_xmls = sorted(Elsevier_xmls, key=lambda x: int(x.split('/')[-1][:-4]))
Elsevier_xmls[:10]

['/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/68.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/114.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/127.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/135.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/141.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/146.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/151.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/155.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/174.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/190.xml']

In [3]:
searching_patterns = [['VOC', 'VOC(V)'], 
                      ['JSC', 'Jsc(mAcm−2)','Avg.Jsc±std(mAcm−2)'], 
                      ['FF', 'FF(%)'],
                      ['PCE', 'PCE(%)', 'Avg.PCE±std(%)', 'Avg.PCE', 'Max.PCE']]
output_headers = ['V OC (V)', 'J SC (mA/cm-2)', 'FF (%)', 'PCE (%)'] 

all_properties = pd.DataFrame()
paper_count = -1

for paper in Elsevier_xmls[:10]:
    
    paper_count += 1
    table_count = -1
    
    for table, table_header in zip(*get_tables_from_xml_file(paper)):
        table_count += 1
        
        properties_table = gather_properties_from_full_table(table, table_header, searching_patterns, output_headers)
        
        if properties_table is None: continue 

        global_idx = []
        for row in range(len(properties_table.index)):
            to_append = str(paper_count) + '_' + str(table_count) + '_' + str(row) #Article Index + Table Index + Row Index 
            global_idx.append(to_append)
            
        properties_table.index = global_idx
        all_properties = pd.concat([all_properties, properties_table], sort=False)

In [4]:
all_properties.head(5)

,0,1,2,V OC (V),J SC (mA/cm-2),FF (%),PCE (%)
0_2_0,In ODCB drop-coating,,,0.82,5.62,42.48,1.96
0_2_1,,PBDTPO-FBT:PC71BM=1:1.5,,0.81,5.80,40.29,1.89
0_2_2,,PBDTPO-FBT:PC71BM=1:2,,0.77,5.63,36.64,1.59
0_2_3,In ODCB with DIO drop-coating,,,0.81,6.72,45.72,2.49
0_2_4,3%,,,0.79,7.23,47.08,2.70
